<a href="https://colab.research.google.com/github/SharuGitHubSpace/E2E-KGQA-DataPipeline/blob/main/GetGoldLabelDataSet_dataPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter
!pip install SPARQLWrapper 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 1.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json 
from urllib.request import urlopen 

In [ ]:
# parameter for urlopen
url_train = r"https://raw.githubusercontent.com/amazon-science/mintaka/main/data/mintaka_train.json"
url_dev = r"https://raw.githubusercontent.com/amazon-science/mintaka/main/data/mintaka_dev.json"
url_test = r"https://raw.githubusercontent.com/amazon-science/mintaka/main/data/mintaka_test.json"

In [ ]:
# callSparql('Q20762698','sing',False)
def callSparql(Q_ID,A_ID,isAnswerEntity):
      import sys
      from SPARQLWrapper import SPARQLWrapper, JSON

      s_p_o_List = []
      triple = []
      Q_ID = Q_ID.replace('Q','wd:Q')
      if isAnswerEntity:
           A_ID = A_ID.replace('Q','wd:Q')
          
      else:
           A_ID = A_ID.lower()
           A_ID = '\'' +  A_ID + '\''


      endpoint_url = "https://query.wikidata.org/sparql"
      query = """
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX foaf: <http://xmlns.com/foaf/0.1/>

      SELECT distinct $s $p $o WHERE
      { 
      {
        VALUES ?s {  """ + Q_ID + """ } .
        FILTER (lcase(str(?o)) IN ("""+ A_ID +""") )
        FILTER(STRSTARTS(STR(?p), "http://www.wikidata.org/prop/direct/"))
        FILTER (lang(?o) = "en")
        $s $p $o
      }
      UNION
      {
          VALUES ?s { """ + Q_ID + """ } .    
          VALUES ?o { """ + A_ID + """ } .
        FILTER(STRSTARTS(STR(?p), "http://www.wikidata.org/prop/direct/"))
        BIND( REPLACE( ?string,"prop/direct/","entity/" ) AS ?entity ).
        $s $p $o
      }
      }
      ORDER BY DESC(?s)
      """

      results = get_results(endpoint_url, query)
      
      for result in results["results"]["bindings"]:
          #print(result)
          #print(result['s']['value'])
          #print(result['p']['value'])
          #print(result['o']['value'])
          s = (result['s']['value']).replace('http://www.wikidata.org/entity/', '')
          p = (result['p']['value']).replace('http://www.wikidata.org/prop/direct/', '')
          o = (result['o']['value']).replace('http://www.wikidata.org/entity/', '')
          triple.append((s , p , o ))
          #print(triple)
          #s_p_o_List.append(triple)  
      return triple

In [ ]:
def get_results(endpoint_url, query):
      import sys
      from SPARQLWrapper import SPARQLWrapper, JSON
      #print(query)
      user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
      # TODO adjust user agent; see https://w.wiki/CX6
      sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
      sparql.setQuery(query)
      sparql.setReturnFormat(JSON)
      return sparql.query().convert()

In [ ]:
# Steps to get the Goldendataset for Model training - Trial 4
# Step 1 : Split the Train(1750)  - Val(250) - Test(500) 
          # conditions are category  = movies , keep questions only that are in answer entity type 'entity' and 'string'
        ###  ###  ###  ###  ###  ###  LOOP             ###  ###  ###  ###
# Step 2 : Follow steps to get the DS for Set 1 
           # loop every question 
           # Extract all QID from question 
           # Extract all Answer id from question
           # write to a file QID-AID-Log.xslx (With reference to train / val / test) every qid - aid , if there are 2 qid make two rows Q_QID1 - A_QID1 ,            
           # Log all combinations
           
# Step 3 : SPARQL - Find if the QID -AID pair has a  predicates from SPARQL
           # if there is match , put the QID-AID pair and s-p-o in the triplet_csv.xslx
           # Copy the QID1 and QID2 and AID to entity_csv.xslx
           # copy the predicate to predicate_csv.xlsx
           # Outcome of this step is the S -> P -> O triplets triplet_csv_master.xlsx

# Step 4 : # From last step map literals to the unique QID and remove duplicates  call it literal_csv.csv

# Step 5 : # Prepare the vlookup in the excel 



In [ ]:
# Train - Val - Test Split
train_split = 875
val_split = 125
test_split = 250

topics = ["movies","history","sports","geography"]
answer_filter_types = ['entity','string' , 'date']


urls = [] 
urls.append(url_train)
urls.append(url_dev)
urls.append(url_test)


In [ ]:
dataForDataSet_Train = []
dataForDataSet_Val = []
dataForDataSet_Test = []

Question_Entity_id = []
Answer_Entity_id = []
# final
s_p_o = []
entities = []
entities_Answer = []
predicates = []
Q_A_List = {}
for url in urls:
    response = urlopen(url)
    data_json1 = json.loads(response.read())  
    for i in range(len(data_json1)):
      A_temp = ""
      Q_temp = ""
      triple = ""
      predicate = ""
      Q_temp_list = []
      data_json = data_json1[i]
      isAnswerEntity = False
      isAnswerEntityMultiple = False
      alltuplesFound = True
      A_temp_list = []
      if (data_json["category"] in topics) and (data_json["answer"]["answerType"] in answer_filter_types[0]): # Q_QID , A_QID -- Entities
            # first , get the QID and AID to query SPARQL 
            if not (data_json["answer"].get("answer") is None) and not (data_json["answer"].get("answer") in ['null']): 
                  #print(data_json["answer"].get("answer"))      
                  for k in range(len(data_json["answer"]["answer"])):
                    if k > 1:
                        isAnswerEntityMultiple = True
                    if not (data_json["answer"]["answer"][k] is None):                         
                          # loop for entities
                          if 'name' in str(data_json["answer"]["answer"][k]): 
                              if str(data_json["answer"]["answer"][k]["name"]).startswith("Q"):
                                A_temp = data_json["answer"]["answer"][k]["name"]
                                A_temp_list.append(A_temp) 
                                isAnswerEntity = True
                          else:  # literals                      
                                A_temp = data_json["answer"]["answer"][k]
                    Answer_Entity_id.append(A_temp)   # Master list  

                    for j in range(len(data_json['questionEntity'])): # get the question id only if there is an answer                               
                          if str(data_json["questionEntity"][j]["name"]).startswith("Q"):
                              Q_temp  = str(data_json["questionEntity"][j]["name"])
                              Q_temp_list.append(Q_temp)
                              Question_Entity_id.append(Q_temp)  
                              Q_A_List[Q_temp] = A_temp
            # Now take the QID and AID and Query Sparkl see if there is P ID hit ; set a boolean value
            # For every element in Q_temp_list with A_temp
            # call SPARQL
            for question_id in Q_temp_list:
               #print(len(A_temp_list))
               if len(A_temp_list) > 1 :                    
                    for a_qid in A_temp_list:
                        if alltuplesFound:
                          triple = callSparql(question_id, a_qid,isAnswerEntity)
                          if not triple:
                              alltuplesFound = False
                              data_json = ''
                          else : 
                              alltuplesFound = True
                              #predicate =  triple[0][1]                
                              # if there is a predicate then we are going to 
                              # Put the S - P - O in the list
                              s_p_o.append(triple)
                              # Put the Question Entity id in the Entity List entity_csv
                              entities.append(question_id)
                              # Put the Answer in the Answer Entity List 
                              if isAnswerEntity:
                                entities.append(a_qid)
                              else:
                                entities_Answer.append(a_qid)
                              # Put the predicate in the Predicate Entity List predicate_csv
                              for t in triple:                                    
                                 predicate =  t[1]  
                                 predicates.append(predicate)
               else:                      
                      triple = callSparql(question_id, A_temp,isAnswerEntity)
                      #print(triple)
                      #print(data_json)
                      if not triple:
                          alltuplesFound = False
                          pass
                      else :                          
                          alltuplesFound = True                                        
                          # if there is a predicate then we are going to 
                          # Put the S - P - O in the list                          
                          s_p_o.append(triple)
                          # Put the Question Entity id in the Entity List entity_csv
                          entities.append(question_id)
                          # Put the Answer in the Answer Entity List 
                          if isAnswerEntity:
                            entities.append(A_temp)
                          else:
                            entities_Answer.append(A_temp)
                          # Put the predicate in the Predicate Entity List predicate_csv
                          for t in triple:                            
                            predicate =  t[1]  
                            predicates.append(predicate)
                      # final step - take only the question set which has the triple in Wikidata and put in the Master DS
     
            if alltuplesFound:
                if urls.index(url) == 0 :
                  dataForDataSet_Train.append(data_json)   
                elif urls.index(url) == 1 :
                  dataForDataSet_Val.append(data_json)    
                elif urls.index(url) == 2 :
                  dataForDataSet_Test.append(data_json)    

In [ ]:
len(dataForDataSet_Test)

282

In [ ]:

def callSparql_getPredicate(predicateList):
      import sys
      from SPARQLWrapper import SPARQLWrapper, JSON
      import pandas as pd
      import openpyxl
      predicateList =  list(set(predicateList))
      predicateList =  [w.replace('P', 'wdt:P') for w in predicateList]
      PID_SPARQL_QUERY = ' '.join(str(x) for x in predicateList) 
      predicate = []
      endpoint_url = "https://query.wikidata.org/sparql"
      query = """
      SELECT distinct ?wdt ?wdLabel ?wdAltLabel
            {
            VALUES ?wdt { """ + PID_SPARQL_QUERY + """ }
            ?wd wikibase:directClaim ?wdt .
            SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
            }
      """
      print(query)
      results = get_results(endpoint_url, query)
           
      for result in results["results"]["bindings"]:
          #print(result)      
          p = (result['wdt']['value']).replace('http://www.wikidata.org/prop/direct/', '')    
          p_label = (result['wdLabel']['value'])  
                               
          if  'wdAltLabel' in result:  
             p_alias =  (result['wdAltLabel']['value'])     
             predicate.append((p , p_label , p_alias ))  
          else:
             predicate.append((p , p_label , '' ))         
      return predicate

In [ ]:
# Make datasets
with open('train_Movies_history_sports_geo_v1_01042023.txt','w', encoding='utf-8') as f:
  for line in dataForDataSet_Train:
    json.dump(line, f, ensure_ascii=False)
    f.write('\n')
with open('dev_movies_history_sports_geo_v1_01042023.txt','w', encoding='utf-8') as f:
  for line in dataForDataSet_Val:
    json.dump(line, f, ensure_ascii=False)
    f.write('\n')
with open( 'test_movies_history_sports_geo_v1_01042023.txt','w', encoding='utf-8') as f:
  for line in dataForDataSet_Test:
    json.dump(line, f, ensure_ascii=False)
    f.write('\n')   

In [ ]:
import pandas as pd
import openpyxl
list1 = callSparql_getPredicate(predicates)
list1
df = pd.DataFrame( list1 , columns=['p_id', 'p_label', 'p_alias']) 
df.to_excel('predicate_csv.xlsx', sheet_name='new_sheet_name')    


      SELECT distinct ?wdt ?wdLabel ?wdAltLabel
            {
            VALUES ?wdt { wdt:P1889 wdt:P1344 wdt:P156 wdt:P1923 wdt:P20 wdt:P3342 wdt:P115 wdt:P3018 wdt:P463 wdt:P3448 wdt:P27 wdt:P144 wdt:P2341 wdt:P1412 wdt:P451 wdt:P36 wdt:P50 wdt:P725 wdt:P793 wdt:P1029 wdt:P166 wdt:P58 wdt:P102 wdt:P527 wdt:P1411 wdt:P4584 wdt:P210 wdt:P413 wdt:P734 wdt:P530 wdt:P276 wdt:P57 wdt:P735 wdt:P488 wdt:P157 wdt:P1336 wdt:P17 wdt:P6166 wdt:P140 wdt:P186 wdt:P1441 wdt:P6 wdt:P86 wdt:P162 wdt:P495 wdt:P1891 wdt:P361 wdt:P825 wdt:P1444 wdt:P159 wdt:P610 wdt:P466 wdt:P7047 wdt:P1445 wdt:P828 wdt:P1346 wdt:P180 wdt:P1830 wdt:P710 wdt:P1268 wdt:P1165 wdt:P5869 wdt:P840 wdt:P664 wdt:P634 wdt:P287 wdt:P737 wdt:P205 wdt:P937 wdt:P35 wdt:P155 wdt:P674 wdt:P1327 wdt:P647 wdt:P61 wdt:P4791 wdt:P1431 wdt:P921 wdt:P1434 wdt:P800 wdt:P25 wdt:P179 wdt:P726 wdt:P22 wdt:P170 wdt:P175 wdt:P26 wdt:P190 wdt:P3373 wdt:P802 wdt:P631 wdt:P112 wdt:P1366 wdt:P69 wdt:P272 wdt:P559 wdt:P706 wdt:P1376 wdt:P54 wdt:P31

In [ ]:

def callSparql_getEntities(entityList):
      import sys
      from SPARQLWrapper import SPARQLWrapper, JSON
      import pandas as pd
      import openpyxl

      entityList =  list(set(entityList))
      entityList =  [w.replace('Q', 'wd:Q') for w in entityList]
      #QID_SPARQL_QUERY = ' '.join(str(x) for x in entityList) 
      entity = []
      endpoint_url = "https://query.wikidata.org/sparql"
      
      for q_id in entityList:
          
          query = """
                SELECT (?property as ?entity_id) 
                ?propertyLabel 
                  (GROUP_CONCAT(DISTINCT(?altLabel); separator = ", ") AS ?entity_alias) WHERE {
                  VALUES ?property {""" + q_id + """} .     #Concat all the Entity id's here
                  OPTIONAL { ?property skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "en") }
                  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" .}
                }
                GROUP BY ?property ?propertyLabel ?propertyDescription
          """
          #print(query)
          results = get_results(endpoint_url, query)
              
          for result in results["results"]["bindings"]:
              #print(result)      
              e = (result['entity_id']['value']).replace('http://www.wikidata.org/entity/', '')    
              e_label = (result['propertyLabel']['value'])  
                                  
              if  'entity_alias' in result:  
                e_alias =  (result['entity_alias']['value'])     
                entity.append((e , e_label , e_alias ))  
              else:
                entity.append((e , e_label , '' ))       

      return entity

In [ ]:
import pandas as pd
import openpyxl
list1 = callSparql_getEntities(entities)
list1
df = pd.DataFrame( list1 , columns=['e_id', 'e_label', 'e_alias']) 
df.to_excel('entity_csv.xlsx', sheet_name='entities')    

In [ ]:
# Convert all lists to excel 

df3 = pd.DataFrame(entities_Answer)
writer = pd.ExcelWriter( 'entitiesLiterals.xlsx', engine='xlsxwriter')
df3.to_excel(writer, sheet_name='AnswerLiteral', index=False)
writer.save()


In [ ]:
entities_Answer

[]

In [ ]:
# Triplet List csv
spo_list = []
for spo in s_p_o:   
  for x in spo:
    spo_list.append(x)
spo_list

df2 = pd.DataFrame(spo_list , columns=['s_id', 'p_id', 'o_id']) 
writer = pd.ExcelWriter( 'triplet_csv.xlsx', engine='xlsxwriter')
df2.to_excel(writer, sheet_name='triples', index=False)
writer.save()


In [ ]:
test = callSparql('Q20762698','sing1',False)
if not test:
  pass
else :
 test[0][1]